In [10]:
import os
import time
import random
import pickle
import warnings
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch.utils.data import DataLoader

from utils import *
from models import PhyDNet

In [11]:
# ignore warning message
warnings.filterwarnings('ignore')

# fix seed
os.environ['PYTHONHASHSEED'] = str(42)
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [12]:
if torch.cuda.is_available():
    print(f'GPU count: {torch.cuda.device_count()}')
    for i in range(torch.cuda.device_count()):
        print(f'GPU {i}: {torch.cuda.get_device_name(i)}')
        print(torch.cuda.get_device_properties(i))
else:
    print('No GPU')

GPU count: 2
GPU 0: NVIDIA A100-SXM4-40GB
_CudaDeviceProperties(name='NVIDIA A100-SXM4-40GB', major=8, minor=0, total_memory=40337MB, multi_processor_count=108)
GPU 1: NVIDIA A100-SXM4-40GB
_CudaDeviceProperties(name='NVIDIA A100-SXM4-40GB', major=8, minor=0, total_memory=40337MB, multi_processor_count=108)


In [13]:
# data : 2017~2021년 5월 31일까지의 전력 사용량 데이터
# train : 2017~2019년 3년동안의 전력 사용량 데이터(3년)
# val : 2020년 1년동안의 전력 사용량 데이터(1년)
# test : 2021년 5월 31일 6개월 동안의 전력 사용량 데이터(6개월)
elec = np.expand_dims(np.load('./dataset/2d_elec.npy'), axis=1)
elec_x, elec_y, elec_val = make_sequential(elec)

train_X = elec_x[:(365 * 24 * 3)]
train_y = elec_y[:(365 * 24 * 3)]

val_X = elec_x[(365 * 24 * 3):(365 * 24 * 4)]
val_y = elec_y[(365 * 24 * 3):(365 * 24 * 4)]

test_X = elec_x[(365 * 24 * 4):]
test_y = elec_y[(365 * 24 * 4):]

In [14]:
train_dataset = CustomDataset(train_X, train_y)
train_loader = DataLoader(train_dataset, 64, pin_memory=True)

val_dataset = CustomDataset(val_X, val_y)
val_loader = DataLoader(val_dataset, 64, pin_memory=True)

test_dataset = CustomDataset(test_X, test_y)
test_loader = DataLoader(test_dataset, 64, pin_memory=True)

In [15]:
print(f'train_size: {len(train_dataset)}')
print(train_dataset.X.shape)
print(train_dataset.Y.shape)

print(f'val_size: {len(val_dataset)}')
print(val_dataset.X.shape)
print(val_dataset.Y.shape)

print(f'test_size: {len(test_dataset)}')
print(test_dataset.X.shape)
print(test_dataset.Y.shape)

train_size: 26280
(26280, 24, 1, 8, 20)
(26280, 24, 1, 8, 20)
val_size: 8760
(8760, 24, 1, 8, 20)
(8760, 24, 1, 8, 20)
test_size: 3577
(3577, 24, 1, 8, 20)
(3577, 24, 1, 8, 20)


In [16]:
X, _ = next(iter(train_loader))

path = './save_model/PhyDNet/'
check_path(path)

# gpu 0 사용
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
constraints = torch.zeros((49, 7, 7)).to(device)
idx = 0
for i in range(7):
    for j in range(7):
        constraints[idx, i, j] = 1
        idx += 1

phycell = PhyDNet.PhyCell(input_shape=(2, 5), input_dim=64, F_hidden_dims=[49], n_layers=1, kernel_size=(7, 7),
                          device=device)
convcell = PhyDNet.ConvLSTM(input_shape=(2, 5), input_dim=64, hidden_dims=[128, 128, 64], n_layers=3,
                            kernel_size=(3, 3), device=device)
encoder = PhyDNet.EncoderRNN(phycell, convcell, device)

layer  0 input dim  64  hidden dim  128
layer  1 input dim  128  hidden dim  128
layer  2 input dim  128  hidden dim  64


In [17]:
def train_on_batch(input_tensor, target_tensor, encoder, encoder_optimizer, criterion, teacher_forcing_ratio):
    encoder_optimizer.zero_grad()
    # input_tensor : torch.Size([batch_size, input_length, channels, cols, rows])
    input_length = input_tensor.size(1)
    target_length = target_tensor.size(1)
    loss = 0
    for ei in range(input_length - 1):
        encoder_output, encoder_hidden, output_image, _, _ = encoder(input_tensor[:, ei, :, :, :], (ei == 0))
        loss += criterion(output_image, input_tensor[:, ei + 1, :, :, :])

    decoder_input = input_tensor[:, -1, :, :, :]  # first decoder input = last image of input sequence

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    for di in range(target_length):
        decoder_output, decoder_hidden, output_image, _, _ = encoder(decoder_input)
        target = target_tensor[:, di, :, :, :]
        loss += criterion(output_image, target)
        if use_teacher_forcing:
            decoder_input = target  # Teacher forcing    
        else:
            decoder_input = output_image

    # Moment regularization  # encoder.phycell.cell_list[0].F.conv1.weight # size (nb_filters,in_channels,7,7)
    k2m = PhyDNet.K2M([7, 7]).to(device)
    for b in range(0, encoder.phycell.cell_list[0].input_dim):
        filters = encoder.phycell.cell_list[0].F.conv1.weight[:, b, :, :]  # (nb_filters,7,7)     
        m = k2m(filters.double())
        m = m.float()
        loss += criterion(m, constraints)  # constrains is a precomputed matrix   
    loss.backward()
    encoder_optimizer.step()
    return loss.item() / target_length


def trainIters(encoder, nepochs, print_every=10, eval_every=10, name=''):
    train_losses = []
    val_losses = []
    best_mse = float('inf')

    encoder_optimizer = torch.optim.Adam(encoder.parameters(), lr=0.001)
#     encoder_optimizer = torch.optim.AdamW(encoder.parameters(), lr=.001)
    
    criterion = nn.MSELoss()

    for epoch in range(0, nepochs):
        t0 = time.time()
        loss_epoch = 0
        teacher_forcing_ratio = np.maximum(0, 1 - epoch * 0.003)

        for i, out in enumerate(train_loader, 0):
            input_tensor = out[0].to(device)
            target_tensor = out[1].to(device)
            loss = train_on_batch(input_tensor, target_tensor, encoder, encoder_optimizer, criterion,
                                  teacher_forcing_ratio)
            loss_epoch += loss

        train_losses.append(loss_epoch)
        if (epoch + 1) % print_every == 0:
            # print('epoch ', epoch, ' loss ', loss_epoch, ' time epoch ', time.time() - t0)
            print(f'Epoch [{epoch+1}/{nepochs}] {time.time() - t0:.2f}s')

        if (epoch + 1) % eval_every == 0:
            mse, mae = evaluate(encoder, val_loader)
            val_losses.append(mse)
            print(f'loss: {loss_epoch:.4f} | val_loss: {mse:.4f}')
            # encoder_optimizer.step(mse)                   
            # mse.backward()
            # encoder_optimizer.step(mse)                   
            # torch.save(encoder.state_dict(), 'save_model/PhyDNet/encoder_{}.pth'.format(name))
            if mse < best_mse:
                best_mse = mse
                torch.save(encoder.state_dict(), f'save_model/PhyDNet/{name}_{epoch}.pth')
            # torch.save(encoder.state_dict(), 'save_model/PhyDNet/encoder_{}.pth'.format(name))
            
    return train_losses, val_losses


def evaluate(encoder, loader):
    total_mse, total_mae = 0, 0
    t0 = time.time()
    with torch.no_grad():
        for i, out in enumerate(loader, 0):
            input_tensor = out[0].to(device)
            target_tensor = out[1].to(device)
            input_length = input_tensor.size()[1]
            target_length = target_tensor.size()[1]

            for ei in range(input_length - 1):
                encoder_output, encoder_hidden, _, _, _ = encoder(input_tensor[:, ei, :, :, :], (ei == 0))

            decoder_input = input_tensor[:, -1, :, :, :]  # first decoder input= last image of input sequence
            predictions = []

            for di in range(target_length):
                decoder_output, decoder_hidden, output_image, _, _ = encoder(decoder_input, False, False)
                decoder_input = output_image
                predictions.append(output_image.cpu())

            input = input_tensor.cpu().numpy()
            target = target_tensor.cpu().numpy()
            predictions = np.stack(predictions)  # (10, batch_size, 1, 64, 64)
            predictions = predictions.swapaxes(0, 1)  # (batch_size,10, 1, 64, 64)

            mse_batch = np.mean((predictions - target) ** 2, axis=(0, 1, 2)).sum()
            mae_batch = np.mean(np.abs(predictions - target), axis=(0, 1, 2)).sum()
            total_mse += mse_batch
            total_mae += mae_batch

            # for a in range(0,target.shape[0]):
            #     for b in range(0,target.shape[1]):
            #         total_ssim += ssim(target[a,b,0,], predictions[a,b,0,]) / (target.shape[0]*target.shape[1]) 

    # print('eval mse ', total_mse / len(loader), ' eval mae ', total_mae / len(loader), ' time= ', time.time() - t0)
    return total_mse / len(loader), total_mae / len(loader)

In [ ]:
train_losses, val_losses = trainIters(encoder,nepochs=50,print_every=1,eval_every=1, name='best_PhyDNet')

Epoch [1/50] 178.11s
loss: 19.5398 | val_loss: 1.2990
Epoch [2/50] 175.19s
loss: 10.8602 | val_loss: 0.8259
Epoch [3/50] 170.87s
loss: 9.8323 | val_loss: 0.9162
Epoch [4/50] 172.27s
loss: 9.2921 | val_loss: 1.0089
Epoch [5/50] 176.01s
loss: 8.4721 | val_loss: 1.7518
Epoch [6/50] 172.40s
loss: 8.2358 | val_loss: 0.9435
Epoch [7/50] 176.19s
loss: 8.5648 | val_loss: 1.1016
Epoch [8/50] 175.51s
loss: 7.5789 | val_loss: 1.1767
Epoch [9/50] 173.05s
loss: 7.1225 | val_loss: 1.1089
Epoch [10/50] 170.22s
loss: 7.0901 | val_loss: 1.0350
Epoch [11/50] 172.24s
loss: 8.2234 | val_loss: 1.2093
Epoch [12/50] 168.61s
loss: 6.6067 | val_loss: 1.1535
Epoch [13/50] 175.29s
loss: 6.3223 | val_loss: 1.0268
Epoch [14/50] 173.77s
loss: 6.1288 | val_loss: 1.1753
Epoch [15/50] 176.89s
loss: 6.1037 | val_loss: 0.8344
Epoch [16/50] 177.57s
loss: 5.8093 | val_loss: 1.2778
Epoch [17/50] 176.48s
loss: 5.4634 | val_loss: 0.9978
Epoch [18/50] 173.43s
loss: 5.4548 | val_loss: 1.2453
Epoch [19/50] 172.88s
loss: 5.3160 

In [ ]:
with open(path + "train_loss.pkl","wb") as f:
    pickle.dump(train_losses, f)
    
with open(path + "val_loss.pkl","wb") as f:
    pickle.dump(val_losses, f)

In [ ]:
plt.title('PhyDNet')
plt.plot(train_losses, label='train')
plt.plot(val_losses, label='val')
plt.xlabel('epochs')
plt.ylabel('losses(MSE)')
plt.legend()
plt.show()